In [746]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import ast


In [747]:
load_dotenv()
OPENSEA_API = os.getenv('OPENSEA_API_KEY')

In [748]:
def requests_session_with_retries():
    session = requests.Session()
    retries = Retry(total=5,  # Total number of retries to allow.
                    backoff_factor=1,  # A backoff factor to apply between attempts.
                    status_forcelist=[500, 502, 503, 504])  # A set of HTTP status codes that we should force a retry on.
    adapter = HTTPAdapter(max_retries=retries)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

In [749]:
def fetch_api_data_paginated(base_url, headers=None, data_key='asset_events'):
    session = requests_session_with_retries()
    all_data = []
    url = base_url
    
    while url:
        print(f"Requesting URL: {url}")
        response = session.get(url, headers=headers, timeout=30)
       
        if response.status_code == 200:
            data = response.json()
            page_data = data.get(data_key, [])
            all_data.extend(page_data)
            next_cursor = data.get('next', "")
            print(next_cursor)
            if next_cursor:
                url = f"{base_url}?next={next_cursor}"
            else:
                url = None
        else:
            print(f"Failed to fetch data from {url} - Status Code: {response.status_code}")
            break
        
    return pd.json_normalize(all_data)  # Convert the aggregated data to a DataFrame


In [750]:
# Function to fetch data from an API
def fetch_api_data(url, headers=None):
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()  # Return the JSON data directly
    else:
        print(f"Failed to fetch data from {url}")
        return None


In [751]:
# USER CHOOSES COLLECTION
collection_slug = input("Enter the collection: ")
APR = input("Enter your target APR: ")

# API's
get_collection_stats = f"https://api.opensea.io/api/v2/collections/{collection_slug}/stats"
get_collection = f"https://api.opensea.io/api/v2/collections/{collection_slug}"
get_events_by_collection = f"https://api.opensea.io/api/v2/events/collection/{collection_slug}?event_type=sale"
get_nft_by_collection = f"https://api.opensea.io/api/v2/collection/{collection_slug}/nfts"
get_traits = f"https://api.opensea.io/api/v2/traits/{collection_slug}"

headers = {"accept": "application/json", "x-api-key": OPENSEA_API}

collection_stats_json = fetch_api_data(get_collection_stats, headers)
# collection_df = fetch_api_data(get_collection, headers)
# traits_df = fetch_api_data(get_traits, headers)
# 
# events_df = fetch_api_data_paginated(get_events_by_collection, headers, data_key='asset_events')
# nft_df = fetch_api_data_paginated(get_nft_by_collection, headers, data_key='nfts')


Requesting URL: https://api.opensea.io/api/v2/events/collection/boredapeyachtclub?event_type=sale
LWV2ZW50X3RpbWVzdGFtcD0yMDI0LTAzLTExKzA3JTNBMjYlM0E1OSYtZXZlbnRfdHlwZT1zdWNjZXNzZnVsJi1waz0yMDg0NjM3NTA2OQ==
Requesting URL: https://api.opensea.io/api/v2/events/collection/boredapeyachtclub?event_type=sale?next=LWV2ZW50X3RpbWVzdGFtcD0yMDI0LTAzLTExKzA3JTNBMjYlM0E1OSYtZXZlbnRfdHlwZT1zdWNjZXNzZnVsJi1waz0yMDg0NjM3NTA2OQ==
Failed to fetch data from https://api.opensea.io/api/v2/events/collection/boredapeyachtclub?event_type=sale?next=LWV2ZW50X3RpbWVzdGFtcD0yMDI0LTAzLTExKzA3JTNBMjYlM0E1OSYtZXZlbnRfdHlwZT1zdWNjZXNzZnVsJi1waz0yMDg0NjM3NTA2OQ== - Status Code: 400
Requesting URL: https://api.opensea.io/api/v2/collection/boredapeyachtclub/nfts
LXBrPTIzMTQzNzAz
Requesting URL: https://api.opensea.io/api/v2/collection/boredapeyachtclub/nfts?next=LXBrPTIzMTQzNzAz
LXBrPTIzMTQzNjUz
Requesting URL: https://api.opensea.io/api/v2/collection/boredapeyachtclub/nfts?next=LXBrPTIzMTQzNjUz
LXBrPTIzMTQzNTkw
Reque

In [754]:
if collection_stats_json:
    # Directly create a DataFrame for 'total' data
    total_data = pd.DataFrame([collection_stats_json['total']])
    
    # Create a DataFrame for 'intervals' data
    intervals_data = pd.DataFrame(collection_stats_json['intervals'])

In [755]:
total_data

,volume,sales,average_price,num_owners,market_cap,floor_price,floor_price_symbol
0,1.487273e+06,46723,31.8317,5274,176759.757361,15.39,ETH


In [756]:
intervals_data

,interval,volume,volume_diff,volume_change,sales,sales_diff,average_price
0,one_day,866.3253,16.0,0.288992,54,0.421053,16.043061
1,seven_day,7443.0744,243.0,0.980604,421,0.000000,17.679512
2,thirty_day,25397.1528,590.0,0.652736,1213,0.000000,20.937471


In [757]:
# Ensure numeric columns are correctly typed
numeric_cols_total = ['volume', 'sales', 'average_price', 'num_owners', 'market_cap', 'floor_price']
total_data[numeric_cols_total] = total_data[numeric_cols_total].apply(pd.to_numeric, errors='coerce')


In [758]:
total_data.isnull().any()

volume                False
sales                 False
average_price         False
num_owners            False
market_cap            False
floor_price           False
floor_price_symbol    False
dtype: bool

In [759]:
# Convert 'volume_change' and 'sales_diff' to numeric types, handling any errors
numeric_cols_intervals = ['volume', 'volume_diff', 'volume_change', 'sales', 'sales_diff', 'average_price']
intervals_data[numeric_cols_intervals] = intervals_data[numeric_cols_intervals].apply(pd.to_numeric, errors='coerce')

In [760]:
intervals_data.isnull().any()

interval         False
volume           False
volume_diff      False
volume_change    False
sales            False
sales_diff       False
average_price    False
dtype: bool

In [761]:
intervals_data['interval'] = intervals_data['interval'].astype('category')

In [769]:
intervals_data['volume_pct_change'] = intervals_data['volume'].pct_change() * 100
intervals_data['sales_pct_change'] = intervals_data['sales'].pct_change() * 100

In [770]:
intervals_data['volume_pct_change']

0           NaN
1    759.154685
2    241.218580
Name: volume_pct_change, dtype: float64

In [771]:
intervals_data['sales_pct_change']

0           NaN
1    679.629630
2    188.123515
Name: sales_pct_change, dtype: float64

In [772]:
collection_json = fetch_api_data(get_collection, headers)

In [774]:
# Assuming `data` is the JSON response dictionary you received from the API
collection_data = {
    "collection": collection_json["collection"],
    "name": collection_json["name"],
    "description": collection_json["description"],
    "image_url": collection_json["image_url"],
    "banner_image_url": collection_json["banner_image_url"],
    "owner": collection_json["owner"],
    "safelist_status": collection_json["safelist_status"],
    "category": collection_json["category"],
    "is_disabled": collection_json["is_disabled"],
    "is_nsfw": collection_json["is_nsfw"],
    "trait_offers_enabled": collection_json["trait_offers_enabled"],
    "collection_offers_enabled": collection_json["collection_offers_enabled"],
    "opensea_url": collection_json["opensea_url"],
    "project_url": collection_json["project_url"],
    "wiki_url": collection_json["wiki_url"],
    "discord_url": collection_json["discord_url"],
    "telegram_url": collection_json["telegram_url"],
    "twitter_username": collection_json["twitter_username"],
    "instagram_username": collection_json["instagram_username"],
    "total_supply": collection_json["total_supply"],
    "created_date": collection_json["created_date"]
}

# Convert this dictionary to a DataFrame if needed
collection_df = pd.DataFrame([collection_data])


In [792]:
collection_df.drop(['image_url', 'banner_image_url', 'owner', 'safelist_status', 'category', 'is_disabled', 'is_nsfw', 'trait_offers_enabled', 'collection_offers_enabled', 'wiki_url', 'discord_url', 'telegram_url', 'twitter_username', 'instagram_username'], axis=1, inplace=True)

In [793]:
collection_df

,collection,name,description,opensea_url,project_url,total_supply,created_date
0,boredapeyachtclub,Bored Ape Yacht Club,The Bored Ape Yacht Club is a collection of 10...,https://opensea.io/collection/boredapeyachtclub,http://www.boredapeyachtclub.com/,9998,2021-04-22


In [777]:
contracts_df = pd.json_normalize(collection_json, 'contracts')
fees_df = pd.json_normalize(collection_json, 'fees')
payment_tokens_df = pd.json_normalize(collection_json, 'payment_tokens')

In [778]:
rarity_df = pd.DataFrame([collection_json['rarity']])

In [779]:
contracts_df

,address,chain
0,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,ethereum


In [794]:
fees_df.drop(['fee', 'required'], axis=1, inplace=True)

In [795]:
fees_df

,recipient
0,0x0000a26b00c1f0df003000390027140000faa719
1,0xa858ddc0445d8131dac4d1de01f834ffcba52ef1


In [796]:
payment_tokens_df.drop(['address'], axis=1, inplace=True)

In [797]:
payment_tokens_df

,symbol,chain,image,name,decimals,eth_price,usd_price
0,ETH,ethereum,https://raw.seadn.io/files/ETH.svg,Ether,18,1.000000000000000,4044.389999999999873000
1,WETH,ethereum,https://raw.seadn.io/files/WETH.svg,Wrapped Ether,18,1.000000000000000,4041.670000000000073000
2,USDC,ethereum,https://raw.seadn.io/files/USDC.svg,USD Coin,6,0.000247140000000,0.999582000000000000
3,APE,ethereum,https://raw.seadn.io/files/APE.svg,ApeCoin,18,0.000570160000000,2.310000000000000000


In [798]:
rarity_df.drop(['strategy_id', 'strategy_version', 'calculated_at'], axis=1, inplace=True)

In [799]:
rarity_df

,max_rank,tokens_scored
0,10000,10000


In [800]:
collection_df.isnull().any()

collection      False
name            False
description     False
opensea_url     False
project_url     False
total_supply    False
created_date    False
dtype: bool

In [801]:
rarity_df.isnull().any()


max_rank         False
tokens_scored    False
dtype: bool

In [802]:
payment_tokens_df.isnull().any()


symbol       False
chain        False
image        False
name         False
decimals     False
eth_price    False
usd_price    False
dtype: bool

In [803]:
collection_df.isnull().any()


collection      False
name            False
description     False
opensea_url     False
project_url     False
total_supply    False
created_date    False
dtype: bool

In [804]:
fees_df.isnull().any()

recipient    False
dtype: bool

In [805]:
contracts_df.isnull().any()

address    False
chain      False
dtype: bool

In [806]:
merged_df = collection_df.merge(payment_tokens_df, on="common_key", how="left")

KeyError: 'common_key'

In [690]:
events_df['nft_identifier'] = events_df['nft.identifier']

In [691]:
print(events_df['nft_identifier'])

0     4933
1     4162
2     4933
3     1435
4     5726
5     2779
6     8975
7     1367
8     8088
9     2779
10    6947
11    7216
12    4527
13    8329
14    3555
15    1367
16    9514
17    4338
18     147
19    6932
20    1636
21    4858
22     536
23    4855
24    6774
25    3004
26     340
27    3900
28    6935
29    8518
30    8722
31     226
32    7424
33    7782
34    6491
35    7844
36    9084
37     536
38    3827
39    6523
40    8772
41    1807
42    9084
43    8720
44    5662
45    3877
46    6186
47    3967
48    2316
49    1807
Name: nft_identifier, dtype: object


In [639]:
collection_stats_df = collection_stats_df.drop(columns=['intervals'])

# Removing duplicates
collection_stats_df.drop_duplicates(inplace=True)

# Handling missing values for 'total.average_price'
collection_stats_df['total.average_price'].fillna(collection_stats_df['total.average_price'].median(), inplace=True)

# Converting data types for 'total.sales'
collection_stats_df['total.sales'] = collection_stats_df['total.sales'].astype(int)



In [640]:
print(collection_df.columns)


Index(['collection', 'name', 'description', 'image_url', 'banner_image_url',
       'owner', 'safelist_status', 'category', 'is_disabled', 'is_nsfw',
       'trait_offers_enabled', 'collection_offers_enabled', 'opensea_url',
       'project_url', 'wiki_url', 'discord_url', 'telegram_url',
       'twitter_username', 'instagram_username', 'contracts', 'editors',
       'fees', 'payment_tokens', 'total_supply', 'created_date',
       'rarity.strategy_id', 'rarity.strategy_version', 'rarity.calculated_at',
       'rarity.max_rank', 'rarity.tokens_scored'],
      dtype='object')


In [641]:
# Fill missing text fields with an empty string
text_columns = ['description', 'image_url', 'banner_image_url', 'project_url', 
                'wiki_url', 'discord_url', 'telegram_url', 'twitter_username', 
                'instagram_username']
collection_df[text_columns] = collection_df[text_columns].fillna('')

# Convert boolean fields
boolean_columns = ['is_disabled', 'is_nsfw', 'trait_offers_enabled', 'collection_offers_enabled']
collection_df[boolean_columns] = collection_df[boolean_columns].astype(bool)

# Convert 'created_date' to datetime
collection_df['created_date'] = pd.to_datetime(collection_df['created_date'])

In [642]:
# Extract contract addresses as a comma-separated string
collection_df['contract_addresses'] = collection_df['contracts'].apply(lambda x: ', '.join([contract['address'] for contract in x]) if isinstance(x, list) else '')

In [643]:
collection_df['created_date']

0   2021-04-22
Name: created_date, dtype: datetime64[ns]

In [644]:
# Example: Summarizing the fees by calculating the total fees
collection_df['total_fees'] = collection_df['fees'].apply(lambda x: sum(fee['fee'] for fee in x) if isinstance(x, list) else 0)

# Example: Extracting the first payment token's symbol as representative
collection_df['primary_payment_token'] = collection_df['payment_tokens'].apply(lambda x: x[0]['symbol'] if isinstance(x, list) and len(x) > 0 else '')

In [645]:
collection_df['total_fees']

0    3.5
Name: total_fees, dtype: float64

In [646]:
# Check if 'rarity' key exists and extract information, else set to default values
collection_df['rarity.strategy_version'] = collection_df.apply(lambda row: row['rarity'].get('strategy_version') if 'rarity' in row and row['rarity'] else '', axis=1)
collection_df['rarity.calculated_at'] = collection_df.apply(lambda row: pd.to_datetime(row['rarity'].get('calculated_at')) if 'rarity' in row and row['rarity'] else pd.NaT, axis=1)
collection_df['rarity.max_rank'] = collection_df.apply(lambda row: row['rarity'].get('max_rank') if 'rarity' in row and row['rarity'] else 0, axis=1)
collection_df['rarity.total_supply'] = collection_df.apply(lambda row: row['rarity'].get('total_supply') if 'rarity' in row and row['rarity'] else 0, axis=1)



In [647]:
print(collection_df['rarity.total_supply'])

0    0
Name: rarity.total_supply, dtype: int64


In [648]:
print(traits_df.columns)

Index(['categories.Hat', 'categories.Eyes', 'categories.Background',
       'categories.Fur', 'categories.Clothes', 'categories.Mouth',
       'categories.Earring', 'counts.Hat', 'counts.Eyes', 'counts.Background',
       'counts.Fur', 'counts.Clothes', 'counts.Mouth', 'counts.Earring'],
      dtype='object')


In [649]:
def expand_trait_counts(column_name):
    # Extract and expand the dictionary into a DataFrame
    expanded_traits = traits_df[column_name].apply(pd.Series).fillna(0)
    
    # Rename the new columns to reflect the trait they represent
    expanded_traits.columns = [f"{column_name}_{trait}" for trait in expanded_traits.columns]
    
    return expanded_traits


In [650]:
count_columns = [col for col in traits_df.columns if col.startswith('counts.')]

for col in count_columns:
    expanded_traits = expand_trait_counts(col)
    traits_df = pd.concat([traits_df.drop(columns=[col]), expanded_traits], axis=1)


In [651]:
print(traits_df)

  categories.Hat categories.Eyes categories.Background categories.Fur  \
0         string          string                string         string   

  categories.Clothes categories.Mouth categories.Earring  \
0             string           string             string   

   counts.Hat_Commie Hat  counts.Hat_Bayc Flipped Brim  \
0                    304                           231   

   counts.Hat_Fisherman's Hat  ...  counts.Mouth_Bored Dagger  \
0                         345  ...                         49   

   counts.Mouth_Bored Unshaven Kazoo  counts.Mouth_Bored Unshaven Party horn  \
0                                 61                                      45   

   counts.Mouth_Bored Unshaven Pizza  counts.Earring_Silver Hoop  \
0                                 26                         882   

   counts.Earring_Gold Stud  counts.Earring_Cross  counts.Earring_Gold Hoop  \
0                       439                   149                       462   

   counts.Earring_Silver St

In [652]:
# Convert all newly created columns (or the entire DataFrame) to the appropriate types
traits_df = traits_df.astype({col: 'int' for col in traits_df.columns if col.startswith('counts.')})

# Verify the structure and contents of the updated DataFrame
print(traits_df.head())


  categories.Hat categories.Eyes categories.Background categories.Fur  \
0         string          string                string         string   

  categories.Clothes categories.Mouth categories.Earring  \
0             string           string             string   

   counts.Hat_Commie Hat  counts.Hat_Bayc Flipped Brim  \
0                    304                           231   

   counts.Hat_Fisherman's Hat  ...  counts.Mouth_Bored Dagger  \
0                         345  ...                         49   

   counts.Mouth_Bored Unshaven Kazoo  counts.Mouth_Bored Unshaven Party horn  \
0                                 61                                      45   

   counts.Mouth_Bored Unshaven Pizza  counts.Earring_Silver Hoop  \
0                                 26                         882   

   counts.Earring_Gold Stud  counts.Earring_Cross  counts.Earring_Gold Hoop  \
0                       439                   149                       462   

   counts.Earring_Silver St

In [653]:
print(events_df.columns)


Index(['event_type', 'order_hash', 'chain', 'protocol_address', 'closing_date',
       'quantity', 'seller', 'buyer', 'transaction', 'event_timestamp',
       'nft.identifier', 'nft.collection', 'nft.contract',
       'nft.token_standard', 'nft.name', 'nft.description', 'nft.image_url',
       'nft.metadata_url', 'nft.opensea_url', 'nft.updated_at',
       'nft.is_disabled', 'nft.is_nsfw', 'payment.quantity',
       'payment.token_address', 'payment.decimals', 'payment.symbol'],
      dtype='object')


In [654]:
# Convert 'event_timestamp' and 'closing_date' to datetime format
events_df['event_timestamp'] = pd.to_datetime(events_df['event_timestamp'], unit='s')  # Assuming timestamp is in seconds
events_df['closing_date'] = pd.to_datetime(events_df['closing_date'], unit='s')  # Adjust unit based on the actual data

events_df.drop(['description', 'seller', 'buyer'], axis=1, inplace=True, errors='ignore')

# Ensure 'quantity' is treated as numeric
events_df['quantity'] = pd.to_numeric(events_df['quantity'], errors='coerce').fillna(0)


In [675]:
def expand_nft_events(events_df, nested_column_name):
    # Extract and expand the dictionary into a DataFrame
    expanded_nft_events = events_df[nested_column_name].apply(pd.Series).fillna(0)
    
    # events_df = events_df.drop(columns=[nested_column_name])
    
    # Rename the new columns to reflect the trait they represent
    # expanded_nft_events.columns = [f"{column_name}_{info}" for info in expanded_nft_events.columns]
    
    # return pd.concat([events_df, expanded_nft_events], axis=1)
    
    events_df = pd.concat([events_df.drop(columns=[nested_column_name]), expanded_nft_events], axis=1)
    
    return events_df

In [676]:
# def change_nested_columns(events_df):
#     
#     for column in events_df.columns:
#         if column.startswith('nft'):
#             new_name = 'new_' + column.split('.')[1]  # Assuming you want to add ‘new_’ prefix
#             events_df.rename(columns={column: new_name}, inplace=True)

In [677]:
nft_event_columns = [col for col in events_df.columns if col.startswith('nft.')]

print(nft_event_columns)



AttributeError: 'int' object has no attribute 'startswith'

In [678]:
events_df = expand_nft_events(events_df, 'nft')

KeyError: 'nft'

In [580]:
# Example of expanding the 'nft' nested fields into separate columns
for nft_field in ['identifier', 'collection', 'contract', 'token_standard', 'name', 'description', 'image_url', 'metadata_url', 'opensea_url', 'updated_at', 'is_disabled', 'is_nsfw']:
    events_df[f'nft.{nft_field}'] = events_df['nft'].apply(lambda x: x.get(nft_field, ''))

# Similar expansion for 'payment' nested fields
for payment_field in ['quantity', 'token_address', 'decimals', 'symbol']:
    events_df[f'payment.{payment_field}'] = events_df['payment'].apply(lambda x: x.get(payment_field, ''))


KeyError: 'nft'